Step 1: Import Libraries

In [7]:
import torch

import torch.nn as nn

import torch.optim  as optim

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

Step 2: Generate Sample Data

We create a simple dataset for classification.

In [8]:
# Create synthetic dataset (binary classification problem)

torch.manual_seed(42)

X = torch.randn(1000, 2)

y = (X[:,0] * X[:,1] > 0).float().unsqueeze(1) # XOR-like pattern

X_train, X_test , y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

Step 3: Define Neural Networks (With and Without Dropout)

In [9]:
# Neural network WITHOUT Dropout

class NeuralNet_NoDropout(nn.Module):

    def __init__(self):
        super(NeuralNet_NoDropout, self).__init__()

        self.fc1 = nn.Linear(2, 128)

        self.fc2 = nn.Linear(128, 64)

        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()

        self.sigmoid = nn.Sigmoid()

    def forward(self,x):

        x = self.relu(self.fc1(x))

        x = self.relu(self.fc2(x))

        x = self.sigmoid(self.fc3(x))

        return x

In [10]:
# Neural network WITH Dropout

class NeuralNet_Dropout(nn.Module):

    def __init__(self):
        super(NeuralNet_Dropout, self).__init__()

        self.fc1 = nn.Linear(2, 128)

        self.fc2 = nn.Linear(128, 64)

        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()

        self.sigmoid = nn.Sigmoid()

        self.dropout = nn.Dropout(0.5) # Dropout layer (50% neurons randomly deactivated)

    def forward(self,x):

        x = self.relu(self.fc1(x))

        x = self.dropout(x) # Apply dropout

        x = self.relu(self.fc2(x))

        x = self.dropout(x) # Apply dropout

        x = self.sigmoid(self.fc3(x))

        return x

Step 4: Train Both Models

In [11]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=200, lr=0.1):

    optimizer = optim.Adam(model.parameters(), lr=lr)

    loss_fn = nn.BCELoss # Binary Cross-Entropy Loss

    train_losses , test_losses = [], []


    for epoch in range(epochs):

        model.train()

        optimizer.zero_grad()

        y_pred = model(X_train)

        loss = loss_fn(y_pred, y_train)

        loss.backward()

        optimizer.step()

        train_losses.append(loss.item())


        # Evaluate on test set

        model.eval()

        with torch.no_grad():

                y_pred_test = model(X_test)

                test_loss = loss_fn(y_pred_test, y_test)

                test_losses.append(test_loss.item())

        if epoch % 50 == 0:
             
            print(f'Epoch: {epoch}/{epochs} - Train loss: {train_losses:.4f} - Test Loss : {test_loss:.4f}')

    
    return train_losses, test_losses



RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
# Train both models

model_no_dropout = NeuralNet_NoDropout()

model_dropout = NeuralNet_Dropout()

train_loss_no_dropout, test_loss_no_dropout = train_model(model_no_dropout, X_train, y_train ,X_test, y_test)

train_loss_dropout, test_loss_dropout = train_model(model_dropout, X_train, y_train, X_test, y_test)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous